In [1]:
import rlcard
import torch
import pathlib
import numpy
import ray
import models
import ddz

import self_play_test
import time
from rlcard.agents import RandomAgent
from rlcard.utils import set_seed, tournament

In [2]:
config = ddz.MuZeroConfig()
observation_shapes = config.observation_shapes
configs = []
for i in range(config.num_players):
    config = ddz.MuZeroConfig()
    config.observation_shape = observation_shapes[i]
    configs.append(config)

In [3]:
agents = []
for i in range(config.num_players):
    model = models.MuZeroNetwork(configs[i])
#     model = model.cuda()
    model.eval()
    agents.append(model)


In [4]:
def setup_seed(seed):
     torch.manual_seed(seed)
#      torch.cuda.manual_seed_all(seed)
     numpy.random.seed(seed)
#      torch.backends.cudnn.deterministic = True

landlord_paths = '/home/yg/code/Muzero//muzero/results/test_ddz/new/model0.checkpoint'
farmer0_paths = '/home/yg/code/Muzero/muzero/results/test_ddz/new/model1.checkpoint'
farmer1_paths = '/home/yg/code/Muzero/muzero/results/test_ddz/new/model2.checkpoint'
new_model = [landlord_paths, farmer0_paths, farmer1_paths]

landlord_paths_o = '/home/yg/code/Muzero//muzero/results/test_ddz/old/model0.checkpoint'
farmer0_paths_o = '/home/yg/code/Muzero/muzero/results/test_ddz/old/model1.checkpoint'
farmer1_paths_o = '/home/yg/code/Muzero/muzero/results/test_ddz/old/model2.checkpoint'
old_model = [landlord_paths_o, farmer0_paths_o, farmer1_paths_o]
order = [[new_model[0],new_model[1],new_model[2]],[new_model[0],old_model[1],old_model[2]],
         [old_model[0],new_model[1],new_model[2]],[old_model[0],old_model[1],old_model[2]]]

four_res = []
for i in range(len(order)):
    models_paths = order[i]

In [ ]:
for i in range(3):
    model_path = pathlib.Path(models_paths[i])
    model_checkpoint = torch.load(model_path, map_location = torch.device('cpu'))
    agents[i].set_weights(model_checkpoint['weights'])

In [4]:
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     numpy.random.seed(seed)
     torch.backends.cudnn.deterministic = True

landlord_paths = '/home/yg/code/Muzero//muzero/results/test_ddz/new/model0.checkpoint'
farmer0_paths = '/home/yg/code/Muzero/muzero/results/test_ddz/new/model1.checkpoint'
farmer1_paths = '/home/yg/code/Muzero/muzero/results/test_ddz/new/model2.checkpoint'
new_model = [landlord_paths, farmer0_paths, farmer1_paths]

landlord_paths_o = '/home/yg/code/Muzero//muzero/results/test_ddz/old/model0.checkpoint'
farmer0_paths_o = '/home/yg/code/Muzero/muzero/results/test_ddz/old/model1.checkpoint'
farmer1_paths_o = '/home/yg/code/Muzero/muzero/results/test_ddz/old/model2.checkpoint'
old_model = [landlord_paths_o, farmer0_paths_o, farmer1_paths_o]
order = [[new_model[0],new_model[1],new_model[2]],[new_model[0],old_model[1],old_model[2]],
         [old_model[0],new_model[1],new_model[2]],[old_model[0],old_model[1],old_model[2]]]

four_res = []
for i in range(len(order)):
    models_paths = order[i]

# models_paths = [landlord_paths,farmer0_paths,farmer1_paths]
# map_location = torch.device('cpu')
    for i in range(3):
        model_path = pathlib.Path(models_paths[i])
        model_checkpoint = torch.load(model_path, map_location = torch.device('cpu'))
        agents[i].set_weights(model_checkpoint['weights'])
        
    seed = 42
    setup_seed(seed)
    num_test = 100
    env = ddz.Game(seed)
    env.set_agents(agents)
    
    self_play_worker = self_play_test.SelfPlayTest(env, config, configs, seed, agents)

    results = []
    for i in range(num_test):
        start_time = time.time()
        results.append(
                self_play_worker.play_game(
                    0,
                    0,
                    False,
                    "self",
                    0,
                )
        )
        end_time = time.time()
        print('time',end_time-start_time, i)
    
    res_last = []
    result = [[],[],[]]
    for game_historys in results:
        for i in range(3):
            result[i].append(
                sum(
                    reward
                    for k, reward in enumerate(game_historys[i].reward_history)
                )
            )
    for i in range(3):
        res_last.append(numpy.mean(result[i]))
    print(res_last)
    four_res.append(res_last)

set_seed------------
{'hand_cards_with_suit': ['D4', 'C6 H6 CJ SJ', ''], 'hand_cards': ['4', '66JJ', ''], 'trace': [(0, 'AAA2'), (1, 'pass'), (2, 'pass'), (0, '5888'), (1, '3TTT'), (2, 'pass'), (0, 'pass'), (1, 'QQ'), (2, 'KK'), (0, 'pass'), (1, 'pass'), (2, '6'), (0, 'R'), (1, 'pass'), (2, 'pass'), (0, '55'), (1, 'pass'), (2, 'pass'), (0, '2'), (1, 'pass'), (2, 'pass'), (0, '9'), (1, 'Q'), (2, 'A'), (0, 'pass'), (1, 'B'), (2, 'pass'), (0, 'pass'), (1, '6'), (2, '8'), (0, 'J'), (1, 'pass'), (2, 'pass'), (0, 'Q'), (1, 'pass'), (2, '2'), (0, 'pass'), (1, 'pass'), (2, '4'), (0, '9'), (1, 'pass'), (2, 'J'), (0, 'K'), (1, 'pass'), (2, '2'), (0, 'pass'), (1, 'pass'), (2, '7'), (0, 'pass'), (1, 'pass'), (2, '99'), (0, 'pass'), (1, 'pass'), (2, '3'), (0, 'pass'), (1, '7'), (2, 'T'), (0, 'pass'), (1, 'K'), (2, 'pass'), (0, 'pass'), (1, '4'), (2, '5'), (0, 'pass'), (1, 'pass'), (2, '3'), (0, '7'), (1, 'pass'), (2, 'pass'), (0, '3'), (1, '4'), (2, '7')], 'current_player': 0, 'legal_actions': []}


KeyboardInterrupt: 